In [1]:
from sconf import Config
from utils import pprint_config
import copy
import numpy as np

format = "format.yaml"
format = Config(format)
pprint_config(format)

> ticker: FOUR
> sector: financials
> eps_growth_0: 30.12
> eps_growth_1: 29.23
> eps_growth_2: 25.0
> eps_growth_3: None
> pe_fwd: 18.87
> peg_ratio_sector: 3.35
> price_current: None


In [2]:
data = copy.deepcopy(format)

In [3]:
import yfinance as yf

class calcTargetPrice(object):
    def __init__(self, data):
        self.data = data
        self.yf_data = yf.Ticker(self.data.ticker)
        if not data.price_current:
            self._get_current_price()
        self._get_52high()
        self._calc_eps_avg()
        self._calc_peg_ratio()
        self._calc_growth_value()
        self._calc_dec_from_52high()
        self._calc_reasonable_price()
        self._calc_target_prices()
        
    def pprint_data(self):
        pprint_config(self.data)
    
    def _get_current_price(self):
        self.data.price_current = self.yf_data.info["regularMarketPreviousClose"]
        
    def _get_52high(self):
        self.data.price_52high = self.yf_data.info["fiftyTwoWeekHigh"]
        
    def _calc_eps_avg(self):
        val_eps_growth = list()
        for key, value in self.data.items():
            if "eps" in key and "avg" not in key and value:
                val_eps_growth.append(value)
        self.data.eps_growth_avg = np.mean(val_eps_growth)
    
    def _calc_peg_ratio(self):
        self.data.peg_ratio = self.data.pe_fwd / self.data.eps_growth_avg
        
    def _calc_growth_value(self):
        if self.data.eps_growth_avg > 0:
            self.data.growth_value = self.data.eps_growth_avg * self.data.peg_ratio_sector
        else:
            self.data.growth_value = (self.data.pe_fwd + (self.data.pe_fwd * self.data.eps_growth_avg)) / 100

    def _calc_dec_from_52high(self):
        self.data.dec_from_52high = (self.data.price_current / self.data.price_52high) - 1
        
    def _calc_reasonable_price(self):
        self.data.price_reasonable = self.data.price_current * (self.data.growth_value / self.data.pe_fwd)
        
    def _calc_target_prices(self):
        if self.data.eps_growth_1:
            self.data.price_target_1 = self.data.price_reasonable * (1 + self.data.eps_growth_1/100)
        if self.data.eps_growth_1 and self.data.eps_growth_2:
            self.data.price_target_2 = self.data.price_reasonable * (1 + self.data.eps_growth_1/100 + self.data.eps_growth_2/100)
        if self.data.eps_growth_1 and self.data.eps_growth_2 and self.data.eps_growth_3:
            self.data.price_target_3 = self.data.price_reasonable * (1 + self.data.eps_growth_1 + self.data.eps_growth_2 + self.data.eps_growth_3)

In [4]:
calc = calcTargetPrice(data=data)

In [5]:
calc.pprint_data()

> ticker: FOUR
> sector: financials
> eps_growth_0: 30.12
> eps_growth_1: 29.23
> eps_growth_2: 25.0
> eps_growth_3: None
> pe_fwd: 18.87
> peg_ratio_sector: 3.35
> price_current: 69.04
> price_52high: 92.3
> eps_growth_avg: 28.116666666666664
> peg_ratio: 0.67113218731476
> growth_value: 94.19083333333333
> dec_from_52high: -0.2520043336944745
> price_reasonable: 344.61765412471294
> price_target_1: 445.34939442536654
> price_target_2: 531.5038079565447
